In [3]:
# import models
import pandas as pd
import numpy as np
import pickle

#pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

#models
from sklearn.ensemble import RandomForestClassifier

In [5]:
from project_fraud.data import cleaned_featured_data

In [6]:
df = cleaned_featured_data('~/data/')

**FEATURE SELECTION**

In [1]:
X = df[['TransactionID','card1','card2','addr1','TransactionAmt','card5','D15','C13','D2','D10','D4','weekday','hours',\
           'dist_mean', 'dist_median','dist_mean_rel','dist_median_rel']]
y = df['isFraud']

NameError: name 'df' is not defined

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

X_test_1, X_val, y_test_1, y_val = train_test_split(X_test, y_test, test_size=0.00017, random_state=1)

**PIPELINES**

In [10]:
n = (X.dtypes != 'object')
num_cols = list(n[n].index)
medium_missing_num_cols = []
low_missing_num_cols =[]
for i in num_cols:
    percentage = df[i].isnull().sum() * 100 / len(df[i])
    if percentage < 15:
        low_missing_num_cols.append(i)
    elif percentage >= 15 and percentage <= 60:
        medium_missing_num_cols.append(i)

In [17]:
num_transformer_low = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
num_transformer_medium = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant')),
    ('one_hot', OneHotEncoder())
])
    
preprocessor = ColumnTransformer([
    ('low_num_imputer',num_transformer_low, low_missing_num_cols),
    ('medium_num_imputer', num_transformer_medium, medium_missing_num_cols),
    ('cat_transformer', cat_pipeline, ['weekday','hours'])],
    remainder='drop')

In [18]:
pipeline_rfc = Pipeline([
    ("preprocessor", preprocessor),
    ("rfc", RandomForestClassifier(n_estimators=1500, max_features='auto', max_depth = 50)),
     ])

In [19]:
pipeline_rfc.fit(X_train, y_train)

/Users/gillesfaltz/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('low_num_imputer', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
  ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

**PICKLE FILE**

In [20]:
import pickle

# Export pipeline as pickle file
with open("pickle_website.pkl", "wb") as file:
    pickle.dump(pipeline_rfc, file)

In [7]:
import pickle

# Export pipeline as pickle file

my_pipeline = pickle.load(open("pickle_website.pkl","rb"))

my_pipeline

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('low_num_imputer', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
  ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [8]:
# Export pipeline as pickle file
with open("pickle_website_2.pkl", "wb") as file:
    pickle.dump(my_pipeline, file)

In [5]:
import sys

sys.getsizeof(my_pipeline)

64

In [6]:
del my_pipeline

**CSV WEBSITE**

In [21]:
X_val.to_csv('data_website.csv', index =False, na_rep='nan')